In [1]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib as mpl
import seaborn as sns
import numpy as np
import math
from cycler import cycler

In [2]:
def plot_methods_results_per_gamma(
    all_results: dict[str, dict],
    groups_key: str,
    n_consumers: int,
    n_producers: int,
    n_runs: int,
    k_rec: int,
    save_path,
) -> None:
    sns.set_style("white")
    mpl.rc('font', **{'size': 14})
    plt.rcParams["font.family"] = "Times New Roman"
    n_methods = len(all_results)
    fig, axes = plt.subplots(1, n_methods, figsize=(5*n_methods, 6), dpi=300)


    colors     = sns.color_palette("tab10", n_colors=10)  # 10 variants
    linestyles = ['-', '--']        # 4 variants
    prop_cycle = cycler('color', colors) * cycler('linestyle', linestyles)




    if n_methods == 1:
        axes = [axes]




    for ax, (method_name, results) in zip(axes, all_results.items()):
        # add grid
        ax.grid(visible=True, which='major', axis='both', color='gray', linestyle='--', linewidth=0.5, alpha=0.4)
        ax.set_prop_cycle(prop_cycle)
        z = 1
        max_z = len(results)
        for group_name, group_results in results.items():
            alpha, alpha_results = next(iter(group_results.items()))
            xs, ys, ye = [], [], []
            for gamma, runs in sorted(alpha_results.items()):
                run_means = [np.mean(run) for run in runs]
                xs.append(gamma)
                ys.append(np.mean(run_means))
                ye.append(np.std(run_means) / np.sqrt(len(runs)))

            if group_name == "all":
                ax.plot(xs, ys, color="black", linestyle="dashdot", marker="s", markersize=4,
                        zorder=max_z, label="Mean", linewidth=2)
            else:
                ax.plot(xs, ys, label=group_name.replace("_", " ").capitalize(),
                         zorder=z, linewidth=2, alpha=0.8)
                ax.fill_between(xs, np.array(ys)-np.array(ye), np.array(ys)+np.array(ye),
                                alpha=0.05, zorder=z)
                z += 1

        ax.set_title(method_name.replace("_", " "))

    # centered x‑label moved closer
    fig.supxlabel(
        r"Fraction of best min producer utility guaranteed, $\gamma$",
        x=0.5, y=0.15, ha="center"
    )
    # shift the y‑label further left
    fig.supylabel(
        "Normalized consumer utility",
        x=0.01,
        y=0.6,
        ha="center"
    )

    # collect legend entries
    handles, labels = axes[-1].get_legend_handles_labels()
    n_labels = len(labels)
    n_rows = 4
    n_cols = math.ceil(n_labels / n_rows)
    # place legend lower to avoid overlap
    fig.legend(
        handles, labels, title="Groups",
        loc='lower center', bbox_to_anchor=(0.5, -0.21),
        ncol=n_cols, frameon=False, markerscale=1
    )

    # adjust layout for legend and labels
    fig.tight_layout(rect=[0, 0.18, 1, 0.93])
    # centered title
    #fig.suptitle(
     #   'Consumer–Producer Utility Tradeoff',
      #  x=0.5, y=1, fontsize=16, fontweight='bold'
    #)

    # save to file
    filename = f"results_per_gamma_{groups_key}_{n_consumers}_{n_producers}_{n_runs}_{k_rec}.pdf"
    save_path.mkdir(parents=True, exist_ok=True)
    plt.savefig(save_path / filename, bbox_inches='tight')
